To preprocess csnow data run the following from a terminal

```
conda create -n csnow gdal --yes
conda activate csnow
bash /home/jovyan/Assimilation/scripts/csnow.sh /home/jovyan/shared/data-knuth/csnow
```

In [ ]:
import rasterio
from rasterio.plot import show
import contextily as ctx
import matplotlib.pyplot as plt 
from topolib import gda_lib;
import numpy as np

### import DTM

In [ ]:
#import raster 
raster_path = '/home/jovyan/shared/data-aragon/ASO_3M_PCDTM_USCATE_20191010_20191010.tif'
topo_raster = rasterio.open(raster_path)
ASOdem = topo_raster.read(1)
ASOdem = np.ma.masked_equal(ASOdem,gda_lib.get_ndv(topo_raster))
print(topo_raster.crs)

In [ ]:
print(np.min(ASOdem),np.max(ASOdem))
show(ASOdem);

In [ ]:
#import raster 
raster_path = '/home/jovyan/shared/data-aragon/ASO_3M_PCDTM_USCATE_20191010_20191010.tif'
topo_raster = xr.open_rasterio(raster_path)
topo_raster

### [reproject raster](https://gdal.org/programs/gdalwarp.html)
This is not working for some reason. When I reproject the dtm the min/max values do not make sense??

In [ ]:
# raster_path = '/home/jovyan/shared/data-aragon/ASO_3M_PCDTM_USCATE_20191010_20191010.tif'
# ! gdalwarp -co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER -srcnodata None -dstnodata -9999 -r cubic -t_srs EPSG:4326 {raster_path} /home/jovyan/shared/data-aragon/reference_dem_4326.tif

In [ ]:
ref_dem_path = '/home/jovyan/shared/data-aragon/reference_dem_4326.tif'
topo_raster = rasterio.open(ref_dem_path)
ref_dem = topo_raster.read(1)
ref_dem = np.ma.masked_equal(ref_dem,gda_lib.get_ndv(topo_raster))
print(topo_raster.crs)

In [ ]:
print(np.min(ref_dem),np.max(ref_dem))
show(ref_dem);

#### Get bounding box from reprojected raster

In [ ]:
xmin,ymin,xmax,ymax = topo_raster.bounds
xmin,ymin,xmax,ymax

### Reproject with rioxarray
have to install in terminal:
$ conda install rioxarray

... kernel dies before finishing the reproject

In [1]:
import rioxarray
import rasterio as rio
raster_path = '/home/jovyan/shared/data-aragon/ASO_3M_PCDTM_USCATE_20191010_20191010.tif'
ASOdem = rioxarray.open_rasterio(raster_path)
ASOdem

<xarray.DataArray (band: 1, y: 18653, x: 21623)>
[403333819 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * y            (y) float64 4.235e+06 4.235e+06 ... 4.179e+06 4.179e+06
  * x            (x) float64 2.427e+05 2.427e+05 ... 3.076e+05 3.076e+05
    spatial_ref  int64 0
Attributes:
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        5001.2265625
    STATISTICS_MEAN:           -108.17093864827
    STATISTICS_MINIMUM:        -9999
    STATISTICS_STDDEV:         5071.598850639
    STATISTICS_VALID_PERCENT:  100
    transform:                 (3.0, 0.0, 242715.0, 0.0, -3.0, 4234848.0)
    scale_factor:              1.0
    add_offset:                0.0
    grid_mapping:              spatial_ref

In [2]:
ASOdem.rio.crs

CRS.from_epsg(32611)

In [ ]:
ASOdem4326 = ASOdem.rio.reproject("epsg:4326")
ASOdem4326.rio.crs

In [ ]:
ASOdem4326.rio.to_raster('/home/jovyan/shared/data-aragon/reference_dem_43262.tif')

#### import ICESat-2 data

In [ ]:
from pathlib import Path

# data folder 
data_home03 = Path('/home/jovyan/shared/data-aragon/C-SNOW/ATL03')
data_home06 = Path('/home/jovyan/shared/data-aragon/C-SNOW/ATL06')
                 
# Create folder if it doesn't exist
data_home03.mkdir(exist_ok=True)
data_home06.mkdir(exist_ok=True)

In [ ]:
from icepyx import icesat2data as ipd

short_name = ['ATL03','ATL06']
spatial_extent = [xmin,ymin,xmax,ymax]
date_range = ['2019-10-07','2019-10-09']

region03 = ipd.Icesat2Data(short_name[0], spatial_extent, date_range)
region06 = ipd.Icesat2Data(short_name[1], spatial_extent, date_range)

In [ ]:
#get a list of the available granule IDs that meet your search criteria
print('ATL03 granules\n',region03.avail_granules(ids=True))
print('ATL06 granules\n',region06.avail_granules(ids=True))

In [ ]:
name = 'nina.aragon7'
email = 'aragonch@oregonstate.edu'

# Only download if data folder is empty
if not list(data_home03.glob('*.h5')):
    region03.earthdata_login(name, email)
    region03.download_granules(data_home03)
    
# Only download if data folder is empty
if not list(data_home06.glob('*.h5')):
    region06.earthdata_login(name, email)
    region06.download_granules(data_home06)

#### get dates of downloaded files

In [ ]:
import glob
# glob to list of files
ATL06_list = glob.glob(str(data_home06)+'/*.h5')
ATL06_list

Figure out how to extract the date from the xr or consider pulling the date out of the filename. This will be used to pull the correct c-snow file. 

In [ ]:
import xarray as xr
group = '/ancillary_data/'
data = xr.open_dataset(ATL06_list[0], engine='h5netcdf')
data

#### clip to region of interest

In [ ]:
%%capture

dem_fn_14     = '/home/jovyan/shared/data-knuth/csnow/SD_20181014.nc.tif'
dem_fn_14_out = '/home/jovyan/shared/data-knuth/csnow/SD_20181014_clip.tif'
!gdal_translate -projwin -130.0 55.0 -110.0 40.0 {dem_fn_14} {dem_fn_14_out}

dem_fn_15     = '/home/jovyan/shared/data-knuth/csnow/SD_20181015.nc.tif'
dem_fn_15_out = '/home/jovyan/shared/data-knuth/csnow/SD_20181015_clip.tif'
!gdal_translate -projwin -130.0 55.0 -110.0 40.0 {dem_fn_15} {dem_fn_15_out}

#### plot with basemap

In [ ]:
url = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'

In [ ]:
# dem_file_name = '/home/jovyan/shared/data-knuth/reference_dem_clip_4326.tif'
# rasterio_dataset2 = rasterio.open(dem_file_name)
# show(rasterio_dataset2);

In [ ]:
rasterio_dataset = rasterio.open(dem_fn_14_out)
spatial_extent = rasterio.plot.plotting_extent(rasterio_dataset)
fig, ax = plt.subplots(1,figsize=(10,10))
ax.imshow(rasterio_dataset.read(1, masked=True), extent=spatial_extent)
#ctx.add_basemap(ax, crs='EPSG:4326', source=url)
ax.imshow(rasterio_dataset.read(1, masked=True), extent=spatial_extent)

In [ ]:
rasterio_dataset = rasterio.open(dem_fn_15_out)
spatial_extent = rasterio.plot.plotting_extent(rasterio_dataset)
fig, ax = plt.subplots(1,figsize=(10,10))
ax.imshow(rasterio_dataset.read(1, masked=True), extent=spatial_extent)
ctx.add_basemap(ax, crs='EPSG:4326', source=url)
ax.imshow(rasterio_dataset.read(1, masked=True), extent=spatial_extent)

In [ ]:
import requests

In [ ]:
base_url = 'https://openaltimetry.org/data/api/icesat2/level3a'

In [ ]:
payload =  {'product':'atl06',
            'startDate':'2018-10-14',
#             'endDate':'2020-10-19',
            'minx':str(bounds[0]),
            'miny':str(bounds[1]),
            'maxx':str(bounds[2]),
            'maxy':str(bounds[3]),
            'trackId':'326',
#             'beamName':'gt3r',
#             'beamName':'gt3l',
            'beamName':'gt2r',
#             'beamName':'gt2l',
#             'photonConfidence':'high',
            'outputFormat':'json'}